# Process each RBP

In [1]:
data_dir = "/cellar/users/aklie/data/datasets/Horlacher_HepG2_CLIP/processed/2023_12_29/encode/U2AF2/chr19"

In [ ]:
# RBP of interest, change this to the RBP you're currently processing
rbp = "U2AF2"

# Extract only R2

In [7]:
%%bash -s "$data_dir" "$rbp"
source activate rbpnet

# Change to the directory
cd $1 || { echo "Directory not found"; exit 1; }

# RBP of interest, change this to the RBP you're currently processing
rbp=$2

# File prefix patterns for the RBP
prefixes=("${rbp}_control" "${rbp}_eCLIP")

# Arrays to hold sorted BAM files for later merging
declare -a control_sorted_bams
declare -a eclip_sorted_bams

# Process files for each prefix
for prefix in "${prefixes[@]}"; do

    # Find files matching the prefix
    files=($(ls ${prefix}*.bam 2> /dev/null))

    # Check if files were found
    if [ ${#files[@]} -eq 0 ]; then
        echo "No files found for ${prefix}"
        continue
    fi

    # Process each BAM file
    for file in "${files[@]}"; do
        
        # Define the output filenames
        r2_bam="${file%.bam}.R2.bam"
        sorted_bam="${file%.bam}.R2.sorted.bam"

        # Extract R2 reads, sort, and index
        echo "Processing $file for R2 reads, sorting, and indexing..."
        samtools view -b -f 0x80 -F 0x4 "$file" > "$r2_bam"
        samtools sort "$r2_bam" -o "$sorted_bam"
        samtools index "$sorted_bam"

        # Clean up (optional): Remove intermediate R2 BAM if desired
        rm -f "$r2_bam"
        
        # Add sorted BAM to the respective array for later merging
        if [[ "$prefix" == "${rbp}_control" ]]; then
            control_sorted_bams+=("$sorted_bam")
        else
            eclip_sorted_bams+=("$sorted_bam")
        fi
    done
done

# Merge replicates

In [8]:
%%bash -s "$data_dir" "$rbp"
source activate rbpnet

# Change to the directory
cd $1 || { echo "Directory not found"; exit 1; }

# RBP of interest, change this to the RBP you're currently processing
rbp=$2

# Merge control replicates
if [ ${#control_sorted_bams[@]} -gt 1 ]; then
    echo "Merging control replicates..."
    samtools merge "${rbp}_control_merged.bam" "${control_sorted_bams[@]}"
    samtools sort "${rbp}_control_merged.bam" -o "${rbp}_control_merged.sorted.bam"
    samtools index "${rbp}_control_merged.sorted.bam"
    rm -f "${rbp}_control_merged.bam"
fi

# Merge eCLIP replicates
if [ ${#eclip_sorted_bams[@]} -gt 1 ]; then
    echo "Merging eCLIP replicates..."
    samtools merge "${rbp}_eCLIP_merged.bam" "${eclip_sorted_bams[@]}"
    samtools sort "${rbp}_eCLIP_merged.bam" -o "${rbp}_eCLIP_merged.sorted.bam"
    samtools index "${rbp}_eCLIP_merged.sorted.bam"
    rm -f "${rbp}_eCLIP_merged.bam"
fi

# Extract 5′ read-start coverage for both plus and minus strands

In [ ]:
%%bash -s "$data_dir" "$rbp"
source activate rbpnet

# Change to the directory
cd $1 || { echo "Directory not found"; exit 1; }

# RBP of interest, change this to the RBP you're currently processing
rbp=$2

# Function to extract 5' read-start coverage for plus and minus strands
extract_coverage() {
    local bam_file="$1"
    local prefix="${bam_file%.bam}"

    # Assuming genome file is named "genome.txt" and located in the same directory
    local genome_file="genome.txt"

    # Extract 5' read-start coverage for plus strand
    bedtools genomecov -ibam "$bam_file" -bg -5 -strand + > "${prefix}.pos.bedGraph"

    # Extract 5' read-start coverage for minus strand
    bedtools genomecov -ibam "$bam_file" -bg -5 -strand - > "${prefix}.neg.bedGraph"
}

# After processing each individual BAM file and merged BAM files
for bam_file in "${control_sorted_bams[@]}" "${eclip_sorted_bams[@]}"; do

    # Check if the BAM file exists (important for merged files which might not be created)
    if [[ -f "$bam_file" ]]; then
        echo "Extracting 5' read-start coverage for $bam_file"
        extract_coverage "$bam_file"
    else
        echo "File $bam_file not found, skipping coverage extraction."
    fi
done

# Extract coverage for merged control, if it exists
if [[ -f "${rbp}_control_merged.sorted.bam" ]]; then
    extract_coverage "${rbp}_control_merged.sorted.bam"
fi

# Extract coverage for merged eCLIP, if it exists
if [[ -f "${rbp}_eCLIP_merged.sorted.bam" ]]; then
    extract_coverage "${rbp}_eCLIP_merged.sorted.bam"
fi

# Convert to BigWig

In [ ]:
%%bash -s "$data_dir" "$rbp"
source activate rbpnet

# Change to the directory
cd $1 || { echo "Directory not found"; exit 1; }

# RBP of interest, change this to the RBP you're currently processing
rbp=$2

# Chromsizes
chrom_sizes=/cellar/users/aklie/data/ref/genomes/hg38/GRCh38_EBV.chrom.sizes

# Function to convert bedGraph to bigWig
convert_to_bigwig() {
    local bedgraph_file="$1"
    local bigwig_file="${bedgraph_file%.bedGraph}.bw"

    # Rename 'plus' to 'pos' and 'minus' to 'neg' in the output filename
    bigwig_file=$(echo "$bigwig_file" | sed 's/plus/pos/' | sed 's/minus/neg/')

    echo "Converting $bedgraph_file to $bigwig_file"
    bedGraphToBigWig "$bedgraph_file" "$chrom_sizes" "$bigwig_file"
}

# Convert bedGraph files to bigWig
for bedgraph_file in *.bedGraph; do
    if [[ -f "$bedgraph_file" ]]; then
        convert_to_bigwig "$bedgraph_file"
    else
        echo "File $bedgraph_file not found, skipping conversion."
    fi
done

# Get candidate training regions

In [ ]:
%%bash -s "$data_dir" "$rbp"
source activate rbpnet

# Change to the directory
cd $1 || { echo "Directory not found"; exit 1; }

# RBP of interest, change this to the RBP you're currently processing
rbp=$2

# Define variables for the enriched windows script
script="/cellar/users/aklie/opt/rbpnet/scripts/enriched-windows.py"
annotation="/cellar/users/aklie/data/ref/genomes/hg38/gencode.v40.basic.annotation.chr19.bed"

# Function to find enriched windows
find_enriched_windows() {
    local pos_bw="$1"
    local neg_bw="$2"
    local output_file="${pos_bw%.pos.bw}.crosslink.bed"

    # Construct and run the command
    cmd="python $script $annotation \
--tile-size 100 \
--min-pval 0.01 \
--min-count 15 \
--min-height 5 \
--bigWigPlus $pos_bw \
--bigWigMinus $neg_bw \
--output $output_file \
--threads 32"
    
    echo "Running: $cmd"
    eval $cmd
}

# Run enriched windows finding for eCLIP replicates and merged files
for bw_file in *eCLIP*.pos.bw; do
    # Construct corresponding negative strand BigWig filename
    neg_bw="${bw_file%.pos.bw}.neg.bw"

    # Check if the corresponding negative strand file exists
    if [[ -f "$neg_bw" ]]; then
        echo "Finding enriched windows for $bw_file and $neg_bw"
        find_enriched_windows "$bw_file" "$neg_bw"
    else
        echo "Corresponding negative strand BigWig $neg_bw not found for $bw_file, skipping."
    fi
done

# Clean up

In [ ]:
%%bash -s "$data_dir"
cd $1

# Get rid of everything not named signal.pos.bw, signal.neg.bw, control.pos.bw, control.neg.bw, peaks.crosslink.bed, metadata.tsv
# Array of filenames to keep
keep_files=("signal.pos.bw" "signal.neg.bw" "control.pos.bw" "control.neg.bw" "peaks.crosslink.bed" "metadata.tsv", "files.txt")

# Construct the find command
find_command="find . -maxdepth 1 -type f"
for file in "${keep_files[@]}"; do
    find_command+=" ! -name '$file'"
done

# Append the action to perform
find_command+=" -exec rm {} +"

# Execute the command
echo "Executing: $find_command"
#eval $find_command

# DONE!

---

# Scratch

In [ ]:
%%bash -s "$data_dir" "$rbp"
source activate rbpnet
cd /cellar/users/aklie/data/datasets/Horlacher_HepG2_CLIP/processed/2023_12_29/encode/U2AF2
mkdir chr19
samtools index U2AF2_control_1.bam
samtools view -b U2AF2_control_1.bam chr19 > U2AF2_control_1.chr19.bam
mv *chr19* chr19